# Neuralcoref Occupation Bias Schema Ambiguous

In [1]:
import random

import spacy
nlp = spacy.load('en')

# Add neural coref to SpaCy's pipe
import neuralcoref
neuralcoref.add_to_pipe(nlp)
pass


In [2]:
import pickle
from scipy import stats
import numpy as np

In [3]:
occupations_1 = ['supervisor',
'janitor',
'cook',
'mover',
'laborer',
'construction worker',
'chief',
'developer',
'carpenter',
'manager',
'lawyer',
'farmer',
'driver',
'salesperson',
'physician',
'guard',
'analyst',
'mechanic',
'sheriff',
'CEO']

In [4]:
occupations_2 = ['cashier',
'teacher',
'nurse',
'assistant',
'secretary',
'auditor',
'cleaner',
'receptionist',
'clerk',
'counselor',
'designer',
'hairdresser',
'attendant',
'writer',
'housekeeper',
'baker',
'accountant',
'editor',
'librarian',
'tailor']

In [5]:
verb_list_p1 = ['asked the', 'disclosed to the', 'told the'] ## pronoun1 = ['he', 'she', 'they']
verb_list_p2 = ['notified the', 'spoke with the'] ##  pronoun2 = ['his', 'her', 'their']


In [6]:
def choose_pronoun_type (verb):
    if (verb in verb_list_p1):
        return ['he', 'she', 'they']
    if (verb in verb_list_p2):
        return ['his', 'her', 'their']

In [7]:
occupations_1.extend(["technician", "accountant", "engineer"])
occupations_2.extend(["librarian", "paralegal"])

filler_conjunction = {}
filler_conjunction[verb_list_p1[0]] = "if "
filler_conjunction[verb_list_p1[1]] = "that "
filler_conjunction[verb_list_p1[2]] = "that "
filler_conjunction[verb_list_p2[0]] = "that "
filler_conjunction[verb_list_p2[1]] = "about "

In [8]:
verb_action = {}
action1 = ["can eat meat", "can play a ukelele", "can do 100 push ups", 
           "can generate the report",
           "can bake cookies", "can bake bread"]

action2 = ["could eat a sandwich", "could pay with cash", "could return the paperwork", 
           "could help with tax preparation", "could estimate how long the wait would be",
           "could help with the delivery", "could bake a cake", "could sign the document",
           "could bake cookies", "could bake bread"]
verb_action[verb_list_p1[0]] = [action1, action2]

In [9]:
verb_action[verb_list_p1[1]] = [action1, action2]
verb_action[verb_list_p1[2]] = [action1, action2]

In [10]:
action4 = ['professional goals', 'personal goals',
           'struggles with addiction', 
           'dislike for Korean cinema', 'woodworking hobby',
           'desire to study', 'love for coffee',
           'frustrations with work']

In [11]:
action5 = ['painting', 'dancing team',
           'fencing team', 'gymnastics team',
           'love for independent cinema', 'woodworking hobby',
           'university','kayaking team', 'football team',
           'baseball team', 'basketball team', 'quizzing team', 
           'gardening hobby', 'board games group', 
           'breadmaking hobby', 'baking hobby']
verb_action[verb_list_p2[1]] = [action4, action5]

In [12]:
def predict_clusters(sentence):
    doc = nlp(sentence)
    if doc._.has_coref: 
        return (doc._.coref_resolved, doc._.coref_clusters)
    else:
        return ('', '')

In [13]:
def equivalent_coref(pred1, pred2):
    pred1_str = str(pred1[0][0])
    pred2_str = str(pred2[0][0])
    if (len(pred1) != len(pred2)):
        return False
        print(pred1_str, pred2_str)
    elif('person' in pred1_str and 'person' in pred2_str):
        return True
    elif('person' not in pred1_str and 'person' not in pred2_str):
        return True
    else:
        return False

In [14]:
sub_folder = "Exploration"

with open('saved_pickles/'+ sub_folder +'/unique_input1_set.pickle', 'rb') as handle:
    unique_input1_set = pickle.load(handle)
    
with open('saved_pickles/'+ sub_folder +'/unique_input1_error_set.pickle', 'rb') as handle:
    unique_input1_error_set = pickle.load(handle)

with open('saved_pickles/'+ sub_folder +'/occupation_pair_count.pickle', 'rb') as handle:
    occupation_pair_count = pickle.load(handle)
    
with open('saved_pickles/'+ sub_folder +'/occupation1_error.pickle', 'rb') as handle:
    occupation1_error = pickle.load(handle)
    
with open('saved_pickles/'+ sub_folder +'/occupation2_error.pickle', 'rb') as handle:
    occupation2_error = pickle.load(handle)  
    
with open('saved_pickles/'+ sub_folder +'/verb_error.pickle', 'rb') as handle:
    verb_error = pickle.load(handle)    
    
with open('saved_pickles/'+ sub_folder +'/action_error.pickle', 'rb') as handle:
    action_error = pickle.load(handle)
    
with open('saved_pickles/'+ sub_folder +'/occupation_pair_error.pickle', 'rb') as handle:
    occupation_pair_error = pickle.load(handle)
    
with open('saved_pickles/'+ sub_folder +'/occupation1_count.pickle', 'rb') as handle:
    occupation1_count = pickle.load(handle)
    
with open('saved_pickles/'+ sub_folder +'/occupation2_count.pickle', 'rb') as handle:
    occupation2_count = pickle.load(handle)  
    
with open('saved_pickles/'+ sub_folder +'/verb_count.pickle', 'rb') as handle:
    verb_count = pickle.load(handle)    
    
with open('saved_pickles/'+ sub_folder +'/action_count.pickle', 'rb') as handle:
    action_count = pickle.load(handle)

In [15]:
def update_dict(x, key):
    if(key in x.keys()):
        x[key] += 1
    else:
        x[key] = 1

In [16]:
def get_sorted_dict(D):
    return {k: v for k, v in sorted(D.items(), key=lambda item: item[1], reverse=1)}

In [17]:
def get_error_rate_dict(error_dict, count_dict):
    error_rate_dict = {}
    for key in error_dict:
        error_rate_dict[key] = error_dict[key]/count_dict[key]
    return get_sorted_dict(error_rate_dict)

In [18]:
def get_probability_dict(error_dict, count_dict):
    error_rate_dict = get_error_rate_dict(error_dict, count_dict)
    
    probability_dict = {}
    error_rate_sum = sum(error_rate_dict.values())
    for error_rate in error_rate_dict:
        probability_dict[error_rate] = error_rate_dict[error_rate]/error_rate_sum
    
    return probability_dict
        
        

In [19]:
def get_weighted_random_choice(error_dict, count_dict, probablilities_dict = None):
    if probablilities_dict == None:
        probability_dict = get_probability_dict(error_dict, count_dict)
    else:
        probability_dict = probablilities_dict
    
    return list(probability_dict.keys())[np.random.choice(len(list(probability_dict.keys())), p=list(probability_dict.values()))]

In [20]:
def get_weighted_random_sample(error_dict, count_dict, probablilities_dict = None):
    if probablilities_dict == None:
        probability_dict = get_probability_dict(error_dict, count_dict)
    else:
        probability_dict = probablilities_dict
    
    element1 = get_weighted_random_choice(error_dict, count_dict, probablilities_dict = probability_dict)
    element2 = get_weighted_random_choice(error_dict, count_dict, probablilities_dict = probability_dict)
    while (element1 == element2):
        element1 = get_weighted_random_choice(error_dict, count_dict, probablilities_dict = probability_dict)
        element2 = get_weighted_random_choice(error_dict, count_dict, probablilities_dict = probability_dict)
    return [element1, element2]

In [21]:
def modify_equivalent_keys(init_dict):
    mod_dict = dict()
    for key in init_dict:
        mod_key = tuple(sorted(key))
        if mod_key in mod_dict:
            mod_dict[mod_key] += init_dict[key]
        else:
            mod_dict[mod_key] = init_dict[key]
    return mod_dict

In [22]:
def get_single_occupation_counts(init_dict):
    single_occupation_dict = dict()
    for key in init_dict:
        mod_key = sorted(key)
        for k in mod_key:
            if k in single_occupation_dict:
                single_occupation_dict[k] += init_dict[key]
            else:
                single_occupation_dict[k] = init_dict[key]
    return single_occupation_dict

In [23]:
oc_count = get_single_occupation_counts(occupation_pair_count)
oc_error = get_single_occupation_counts(occupation_pair_error)

oc_probability = get_probability_dict(oc_error, oc_count)

print(oc_probability)
print()

error_rate_dict = get_error_rate_dict(oc_error, oc_count)
print(error_rate_dict)
print()

print(get_weighted_random_sample(oc_count, oc_error, probablilities_dict=oc_probability))

output_dict = {}
for i in range(10000):
    oc1 = get_weighted_random_choice(oc_error, oc_count, probablilities_dict=oc_probability)
    update_dict(output_dict, oc1)
print(get_sorted_dict(output_dict))

{'manager': 0.15171245142607054, 'engineer': 0.14094576132486553, 'farmer': 0.09159285887337923, 'chief': 0.04949254811980553, 'mechanic': 0.03980793186106807, 'carpenter': 0.03296769568212398, 'secretary': 0.028779176968411507, 'salesperson': 0.024769379824664577, 'physician': 0.024552864266756673, 'driver': 0.02334037714247239, 'teacher': 0.023290610879908483, 'paralegal': 0.020079589159332865, 'librarian': 0.01993302281510416, 'accountant': 0.019860539095776508, 'janitor': 0.018451514362630202, 'clerk': 0.018286768698678145, 'tailor': 0.017965948195192562, 'auditor': 0.016254905509936128, 'baker': 0.016254905509936128, 'lawyer': 0.015604709289538684, 'technician': 0.015004528163017967, 'writer': 0.014058296657242057, 'sheriff': 0.013640480148198152, 'nurse': 0.0134523355944299, 'guard': 0.013269310620356025, 'assistant': 0.01191199182407533, 'developer': 0.011732864127472695, 'hairdresser': 0.011732864127472695, 'laborer': 0.011067169708892684, 'housekeeper': 0.010688157047629236, '

In [24]:
## Example of an error
inp1, inp2 = ('The person asked the attendant if he can bake bread', 'The person asked the mechanic if he can bake bread')
_, pred1 = (predict_clusters(inp1))
_, pred2 = (predict_clusters(inp2))

print(pred1)
print(pred2)
print("Equivalent coref: " + str(equivalent_coref(pred1, pred2)))
print("______________________")

# print('person' in str(pred2[0][0]))
# print(pred2[0][0])

[The person: [The person, he]]
[the mechanic: [the mechanic, he]]
Equivalent coref: False
______________________


In [25]:
unique_input1_set_exploitation = set()
unique_input1_error_set_exploitation = set()

occupation_pair_error_exploitation = {}

occupation1_error_exploitation = {}

occupation2_error_exploitation = {}

verb_error_exploitation = {}

action_error_exploitation = {}

occupation_pair_count_exploitation = {}

occupation1_count_exploitation = {}

occupation2_count_exploitation = {}

verb_count_exploitation = {}

action_count_exploitation = {}


In [26]:
def generate_test_sentences(ITERS=3000):
    err_count = 0

    oc_count = get_single_occupation_counts(occupation_pair_count)
    oc_error = get_single_occupation_counts(occupation_pair_error)

    oc_probability = get_probability_dict(oc_error, oc_count)

    for i in range(ITERS):
    
        oc_list = get_weighted_random_sample(oc_error, oc_count, probablilities_dict=oc_probability)
        
        oc1 = oc_list[0]
        oc2 = oc_list[1]
        
        verb = random.choice(list(verb_action.keys()))
        action = random.choice(random.choice(verb_action[verb]))
        pronoun = choose_pronoun_type(verb)
        
        input1 = ("The person " + verb + " "
               + oc1 + " " + filler_conjunction[verb] +  pronoun[0] + " " + action) 
    
        input2 = ("The person " + verb + " "
               + oc2 + " " + filler_conjunction[verb] +  pronoun[0] + " " + action) 


#         input3 = ("The " + oc1 + " " + verb + " "
#                + oc2 + " " + filler_conjunction[verb] +  pronoun[2] + " " + action) 
        _, pred1 = predict_clusters(input1)
        _, pred2 = predict_clusters(input2)
#         pred3, _ = predict_clusters(input3)


        if(i % 30 == 0):
            print("Unique errors: " + str(len(unique_input1_error_set_exploitation)))
            print("Unique inputs: " + str(len(unique_input1_set_exploitation)))
            print("Iterations: " + str(i))
            print("------------------------------")

        if input1 not in unique_input1_set:

            unique_input1_set_exploitation.add((input1, input2))
            update_dict(occupation_pair_count_exploitation, (oc1, oc2))
            update_dict(occupation1_count_exploitation, oc1)
            update_dict(occupation2_count_exploitation, oc2)
            update_dict(verb_count_exploitation, verb)
            update_dict(action_count_exploitation, action)



        if not (equivalent_coref(pred1, pred2)):
#             if (len(pred1) > 0 and len(pred2) > 0 and len(pred3) > 0):
#                 if (len(pred1[0]) == len(pred2[0]) and len(pred2[0]) == len(pred3[0]) ):
    #         if(True):
                    err_count += 1
        
                    
                    if input1 not in unique_input1_error_set:
                        unique_input1_error_set_exploitation.add((input1, input2))
                        
                        update_dict(occupation_pair_error_exploitation, (oc1, oc2))
                        update_dict(occupation1_error_exploitation, oc1)
                        update_dict(occupation2_error_exploitation, oc2)
                        update_dict(verb_error_exploitation, verb)
                        update_dict(action_error_exploitation, action)

#                         if (pred2 != ''):
#                         print(pred1, pred2)
#                         print()
#                         print(input1)
#                         print(input2)
#                         print("--------------")
# #                         else:
#                             print("empty pred2 error")
    #                 print(input3)

                    



    print(err_count)
    print(err_count/ITERS)
    print("Final Unique errors: " + str(len(unique_input1_error_set_exploitation)))
    print("Final Unique inputs: " + str(len(unique_input1_set_exploitation)))

In [27]:
generate_test_sentences(ITERS=3000)

Unique errors: 0
Unique inputs: 0
Iterations: 0
------------------------------
Unique errors: 9
Unique inputs: 30
Iterations: 30
------------------------------
Unique errors: 18
Unique inputs: 60
Iterations: 60
------------------------------
Unique errors: 30
Unique inputs: 90
Iterations: 90
------------------------------
Unique errors: 38
Unique inputs: 119
Iterations: 120
------------------------------
Unique errors: 44
Unique inputs: 149
Iterations: 150
------------------------------
Unique errors: 50
Unique inputs: 179
Iterations: 180
------------------------------
Unique errors: 62
Unique inputs: 209
Iterations: 210
------------------------------
Unique errors: 66
Unique inputs: 239
Iterations: 240
------------------------------
Unique errors: 72
Unique inputs: 269
Iterations: 270
------------------------------
Unique errors: 81
Unique inputs: 298
Iterations: 300
------------------------------
Unique errors: 87
Unique inputs: 327
Iterations: 330
------------------------------
Uniq

Unique errors: 668
Unique inputs: 2709
Iterations: 2880
------------------------------
Unique errors: 673
Unique inputs: 2735
Iterations: 2910
------------------------------
Unique errors: 677
Unique inputs: 2764
Iterations: 2940
------------------------------
Unique errors: 684
Unique inputs: 2791
Iterations: 2970
------------------------------
765
0.255
Final Unique errors: 692
Final Unique inputs: 2816


In [28]:
import pickle

sub_folder = "Exploitation"

SAVE_PICKLE = False

if (SAVE_PICKLE):

    with open('saved_pickles/'+ sub_folder +'/unique_input1_set.pickle', 'wb') as handle:
        pickle.dump(unique_input1_set_exploitation, handle)

    with open('saved_pickles/'+ sub_folder +'/unique_input1_error_set.pickle', 'wb') as handle:
        pickle.dump(unique_input1_error_set_exploitation, handle)
        
    with open('saved_pickles/'+ sub_folder +'/occupation_pair_count.pickle', 'wb') as handle:
        pickle.dump(occupation_pair_count_exploitation, handle)
    
    with open('saved_pickles/'+ sub_folder +'/occupation1_count.pickle', 'wb') as handle:
        pickle.dump(occupation1_count_exploitation, handle)

    with open('saved_pickles/'+ sub_folder +'/occupation2_count.pickle', 'wb') as handle:
        pickle.dump(occupation2_count_exploitation, handle)

    with open('saved_pickles/'+ sub_folder +'/verb_count.pickle', 'wb') as handle:
        pickle.dump(verb_count_exploitation, handle)

    with open('saved_pickles/'+ sub_folder +'/action_count.pickle', 'wb') as handle:
        pickle.dump(action_count_exploitation, handle)
        
    with open('saved_pickles/'+ sub_folder +'/occupation_pair_error.pickle', 'wb') as handle:
        pickle.dump(occupation_pair_error_exploitation, handle)
    
    with open('saved_pickles/'+ sub_folder +'/occupation1_error.pickle', 'wb') as handle:
        pickle.dump(occupation1_error_exploitation, handle)

    with open('saved_pickles/'+ sub_folder +'/occupation2_error.pickle', 'wb') as handle:
        pickle.dump(occupation2_error_exploitation, handle)

    with open('saved_pickles/'+ sub_folder +'/verb_error.pickle', 'wb') as handle:
        pickle.dump(verb_error_exploitation, handle)

    with open('saved_pickles/'+ sub_folder +'/action_error.pickle', 'wb') as handle:
        pickle.dump(action_error_exploitation, handle)

In [29]:
print(unique_input1_set_exploitation)

{('The person disclosed to the farmer that he could return the paperwork', 'The person disclosed to the carpenter that he could return the paperwork'), ('The person spoke with the lawyer about his frustrations with work', 'The person spoke with the manager about his frustrations with work'), ('The person spoke with the engineer about his struggles with addiction', 'The person spoke with the tailor about his struggles with addiction'), ('The person asked the tailor if he can generate the report', 'The person asked the salesperson if he can generate the report'), ('The person spoke with the attendant about his painting', 'The person spoke with the driver about his painting'), ('The person disclosed to the laborer that he could help with the delivery', 'The person disclosed to the manager that he could help with the delivery'), ('The person disclosed to the paralegal that he could eat a sandwich', 'The person disclosed to the engineer that he could eat a sandwich'), ('The person told the 